<h1 style="color:blue">Chapter 8. Data Wrangling: Join, Combine, and Reshape</h1>

In [3]:
import pandas as pd
import numpy as np

<h2 style="color:orange">8.1 Hierarchical Indexing</h2>

In [5]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [6]:
data

a  1    0.815934
   2    0.756067
   3    0.376034
b  1    0.524363
   3    0.943793
c  1    0.629788
   2    0.534917
d  2    0.078852
   3    0.346981
dtype: float64

In [7]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [8]:
data["b"]

1    0.524363
3    0.943793
dtype: float64

In [9]:
data["b":"c"]

b  1    0.524363
   3    0.943793
c  1    0.629788
   2    0.534917
dtype: float64

In [10]:
data.loc[["b", "d"]]

b  1    0.524363
   3    0.943793
d  2    0.078852
   3    0.346981
dtype: float64

In [12]:
data.loc[:, 2]

a    0.756067
c    0.534917
d    0.078852
dtype: float64

In [13]:
data.unstack()

,1,2,3
a,0.815934,0.756067,0.376034
b,0.524363,NaN,0.943793
c,0.629788,0.534917,NaN
d,NaN,0.078852,0.346981


In [14]:
data.unstack().stack()

a  1    0.815934
   2    0.756067
   3    0.376034
b  1    0.524363
   3    0.943793
c  1    0.629788
   2    0.534917
d  2    0.078852
   3    0.346981
dtype: float64

In [27]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[["a", "a", "b", "b"],
                            [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])

In [23]:
frame

Ohio     Colorado
  Green Red    Green
a     0   1        2
a     3   4        5
b     6   7        8
b     9  10       11

In [28]:
frame.index.names = ["key1", "key2"]

In [30]:
frame.columns.names = ["state",  "color"]

In [31]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [32]:
frame.index.nlevels

2

In [33]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [35]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorade"],
                           ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorade', 'Green')],
           names=['state', 'color'])

<h3 style="color:purple">Reordering and Sorting Levels</h3>

In [37]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [38]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [41]:
frame.swaplevel(0, 1).sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [43]:
frame.sort_index()

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

<h3 style="color:purple">Summary Statistics by Level</h3>

In [44]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [47]:
frame.groupby(level="color", axis="columns").sum()

/tmp/ipykernel_13850/775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

<h3 style="color:purple">Indexing with a DataFrame's Columns</h3>

In [50]:
frame = pd.DataFrame({"a": range(7), 
                      "b": range(7, 0 , -1), 
                      "c": ["one", "one", "one", "two", "two", "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

In [51]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [52]:
frame2 = frame.set_index(["c", "d"])

In [53]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [54]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [55]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [56]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

<h2 style="color:orange">8.2 Combining and Merging Datasets</h2>